# 6. Tuning

In [1]:
MODEL = 'CBR'
TUNING_MODEL = 'guanganb'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAnB_H2H

In [11]:
guangan = GuangAnB_H2H.GuangAnB()

GuangAn Initialised


In [12]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00


In [13]:
from catboost import CatBoostRegressor as CBR

In [14]:
guangan.read_in_model(CBR, 'Regression')

Successfully read in model <class 'catboost.core.CatBoostRegressor'>, which is a Regression model


In [15]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [16]:
parameter_choices = {
    'n_estimators': set(range(25, 401)),
    'max_depth': set(range(2, 17)),
    'subsample': (0.5, 0.85),
    'colsample_bylevel': (0.5, 0.85),
    'max_bin': set(range(64, 256)),
    'reg_lambda': (-4, 0),
    'learning_rate': (-4, 0),

}

In [17]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [18]:
guangan.read_in_transform({'learning_rate': '10^', 'reg_lambda': '10^'})

Updated transform dictionary: {'n_estimators': False, 'max_depth': False, 'subsample': False, 'colsample_bylevel': False, 'max_bin': False, 'reg_lambda': '10^', 'learning_rate': '10^'}


In [19]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817, 'verbose': False})

Successfully recorded non_tuneable_hyperparameter choices


In [20]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [21]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 10369 rows
True


In [22]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Has trained 10369 combinations so far
Already Trained and Tested combination {'n_estimators': 307, 'max_depth': 16, 'subsample': 0.675, 'colsample_bylevel': 0.5875, 'max_bin': 160, 'reg_lambda': -4.0, 'learning_rate': -2.0, 'features': 27}, which had val score of 0.5681
        Current best combo: (307, 13, 0.675, 0.5875, 160, -4.0, -1.0, 27) with val score 0.6289. 
        Has trained 10369 combinations so far
Already Trained and Tested combination {'n_estimators': 307, 'max_depth': 16, 'subsample': 0.675, 'colsample_bylevel': 0.5875, 'max_bin': 160, 'reg_lambda': -4.0, 'learning_rate': -2.0, 'features': 35}, which had val score of 0.5786
        Current best combo: (307, 13, 0.675, 0.5875, 160, -4.0, -1.0, 27) with val score 0.6289. 
        Has trained 10369 combinations so far
Already Trained and Tested combination {'n_estimators': 307, 'max_depth': 16, 'subsample': 0.675, 'colsample_bylevel': 0.5875, 'max_bin': 160, 'reg_l

In [ ]:
guangan.view_best_combo_and_score()